In [74]:
import torch
import torch.nn as nn
from unet import Unet
from dataset import RoadDataset
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
#https://www.kaggle.com/datasets/pkdarabi/brain-tumor-image-dataset-semantic-segmentation

In [88]:

# Hyperparameters from command line arguments
learning_rate = 1e-3
batch_size = 3  # Get batch size from command line argument
epochs = 10  # Get number of epochs from command line argument


In [ ]:
# Model setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Unet(input_channel=3,class_num=2)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Dataset setup
data_path='deepglobe-road-extraction-dataset'
dataset_train = RoadDataset(data_path, "train")
dataset_val = RoadDataset(data_path, "test")

loader_train = DataLoader(dataset_train, batch_size=batch_size)
loader_val = DataLoader(dataset_val, batch_size=batch_size)


In [90]:
# Loss function
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

# Training loop
size = len(loader_train.dataset)
model.train()

# Outer loop for epochs
for epoch in range(epochs):
    running_loss = 0.0
    # Training phase
    with tqdm(loader_train, unit="batch", desc=f"Epoch {epoch+1}/{epochs}") as tepoch:
        for batch, (image, label) in enumerate(tepoch):
            image = image.to(device)
            label = label.to(device)

            # Forward pass
            logits = model(image)
            pred_probab = nn.Softmax(dim=1)(logits)
             
            # Compute the loss
            loss = loss_fn(pred_probab, label.squeeze(1))

            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # Accumulate loss for the epoch
            running_loss += loss.item()


    # Average loss for the epoch
    print(f"Epoch [{epoch+1}/{epochs}] Average Loss: {running_loss / size:.4f}")

    # Validation phase (after each epoch)
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():  # No need to compute gradients during validation
        for batch, (image, label) in enumerate(loader_val):
            image = image.to(device)
            label = label.to(device)

            # Forward pass
            logits = model(image)
            pred_probab = nn.Softmax(dim=1)(logits)

            # Compute the loss
            loss = loss_fn(pred_probab, label.squeeze(1))
            val_loss += loss.item()

    # Average validation loss
    print(f"Epoch [{epoch+1}/{epochs}] Validation Loss: {val_loss / len(loader_val):.4f}")

    # Set the model back to training mode
    model.train()
#Preguntas  - Cambiar el ouput de la ultimas capas, usar el test como validation o como un validation final

Epoch 1/10:   0%|          | 0/2076 [00:00<?, ?batch/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacity of 5.77 GiB of which 28.31 MiB is free. Including non-PyTorch memory, this process has 5.73 GiB memory in use. Of the allocated memory 5.52 GiB is allocated by PyTorch, and 115.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [8]:
label.shape

torch.Size([3, 1, 256, 256])